In [ ]:
# Name - Raj Mahajan
# Roll Number - CS22M067
# Assignment 1
# Question 3
# Optimizers combined

In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

In [ ]:
# Load the Fashion-MNIST dataset
(X_train, Y_train), (x_test, y_test) = fashion_mnist.load_data()
X_train=X_train.reshape(X_train.shape[0],784)
x_test=x_test.reshape(x_test.shape[0],784)
Y_train=Y_train.reshape(60000,1)
y_test=y_test.reshape(10000,1)
# X_train.shape
Y_train.shape

(60000, 1)

In [ ]:
# Normalizing the data
X_train=X_train/255.0
x_test=x_test/255.0
y_test.shape

(10000, 1)

In [ ]:
def actual_prob_dist(y) :
  ans=np.zeros((10, 1))
  ans[y]=1
  return(ans)

def sigmoid(a) :
  # s=1/(1+np.exp(a))

  if isinstance(a,np.ndarray):
    return 1.0 / (1.0 + np.exp(-a))
  else :
    if (x>=0):
      return 1.0 / (1.0 + np.exp(-a))
    else :
      return np.exp(a) / ( 1.0 + np.exp(a))
  # return(s)



def derivative_sigmoid(x):
  ans = sigmoid(x) * (1 -sigmoid(x))
  return(ans)


def softmax(v):
  # ans = np.exp(v) / np.sum(np.exp(v), axis=0)
  z = v - max(v)
  numerator = np.exp(z)
  denominator = np.sum(numerator)
  ans = numerator/denominator
  return(ans)


def derivative_softmax(x):
  ans= softmax(x) * (1-softmax(x))
  return(ans)


def cross_entropy(y, yhat, batch_size):
   loss = (-1.0 * np.sum(np.multiply(y, np.log(yhat))))/batch_size
   return(loss)

In [ ]:
def add_list_grad(list_grad_x , list_grad_y):
  if(len(list_grad_x) == 0 ):
    return(list_grad_y)
  for i in range(0,len(list_grad_x)):
    list_grad_x[i] += list_grad_y[i]
  return(list_grad_x)

def add_list_grad_eta(list_grad_x , list_grad_y, eta):
  if(len(list_grad_x) == 0 ):
    return(list_grad_y)
  for i in range(0,len(list_grad_x)):
    list_grad_x[i] -= eta * list_grad_y[i].transpose()
  return(list_grad_x)

def initilize_theta_random(num_layers,num_neurons):
  theta=[]
  i=0
  temp=np.random.randn(28 * 28, num_neurons)
  theta.append(temp)
  while i< num_layers-1 :
    temp=np.random.randn(num_neurons, num_neurons)
    theta.append(temp)
    i+=1
  temp=np.random.randn(num_neurons, 10) 
  theta.append(temp)
  return(theta)

def initilize_theta_xavier(num_layers,num_neurons):
  theta=[]
  i=0
  temp=np.random.randn(28 * 28, num_neurons) * np.sqrt(2/(28 * 28 + num_neurons)) # xavier
  theta.append(temp)
  while i< num_layers-1 :
    temp=np.random.randn(num_neurons, num_neurons) * np.sqrt(2/(num_neurons + num_neurons)) # xavier
    theta.append(temp)
    i+=1
  temp=np.random.randn(num_neurons, 10) * np.sqrt(2/(num_neurons + 10)) # xavier
  theta.append(temp)
  return(theta)

def initilize_bias(num_layers,num_neurons):
  b=[]
  i=0
  while i < num_layers :
    temp=np.zeros((1, num_neurons))
    b.append(temp)
    i+=1
  temp=np.zeros((1, 10))
  b.append(temp)
  return(b)


def image_pridiction(x_test, y_test, theta, b, num_layers,activation):
  img_no = 0
  wrong_count = 0
  while(img_no<len(x_test)):
    img = x_test[img_no].reshape(784,1)
    clas = y_test[img_no]
    temp_ans = forward_prop(activation,theta,b,num_layers,img)
    yhat = temp_ans[0]
    temp = -1
    predicted_class = -1
    # print(yhat.shape)
    sum=0
    for i in range(0 , 10):
      sum+=yhat[i]
      if(temp < yhat[i]):
        predicted_class = i
        temp = yhat[i]
    # print(sum)
    # print(predicted_class)
    if(clas != predicted_class):
      wrong_count+=1
    img_no+=1
  return(wrong_count)

In [ ]:
def activation_fun(activation,a):
  if(activation == "sigmoid"):
    return(sigmoid(a))
  

In [ ]:
# Constructed only for 1 image 
# h will be image input if 1d vector of length 28x28
def forward_prop(activation,theta,b,num_layers,h):
  h_list=[]
  h_list.append(h)
  a_list=[]
  for k in range(0,num_layers):
    a = b[k].transpose() + np.matmul(theta[k].transpose() , h)
    a_list.append(a)
    h= activation_fun(activation,a)
    h_list.append(h)
  # For last output layer
  a = b[num_layers].transpose() + np.matmul(theta[num_layers].transpose() , h)
  a_list.append(a)
  yhat = softmax(a)
  return(yhat,h_list,a_list)


In [ ]:
# only for 1 image
def backward_prop(theta,h_list,a_list,y,yhat,num_layers):
  grad_a=-(actual_prob_dist(y) - yhat)
  list_grad_w=[]
  list_grad_b=[]
  for k in range(num_layers , -1,-1) :
    grad_w = np.matmul(grad_a , h_list[k].transpose())
    list_grad_w.append(grad_w)
    grad_b = grad_a.copy()
    list_grad_b.append(grad_b)
    if k >0:
      grad_h = np.matmul(theta[k] , grad_a)
      grad_a = grad_h * derivative_sigmoid(a_list[k-1])
  return(list_grad_w , list_grad_b)
  

# grad_b.shape = 32 x 1
# grad_w.shape = 32 x 32
# grad_a.shape = 10/32 x 1

In [ ]:
def SGD(theta,b,epochs,eta,actiavtion,x_train,y_train,num_neurons, num_layers,batch_size):
  t=0
  while t<epochs :
    t += 1
    print(t)
    image_no = 0
    list_grad_w = []
    list_grad_b = []
    loss = 0
    while(image_no<len(x_train)):
         
      # forward propgation
      h=x_train[image_no].reshape(784,1)
      y_hat,h_list,a_list = forward_prop(activation,theta,b,num_layers,h)
      # backward propogation
      y = y_train[image_no]
      if( y_hat[y]!= 0):
        loss += (-math.log(y_hat[y]))
      bp_list = backward_prop(theta,h_list,a_list,y_train[image_no],y_hat,num_layers)
      list_grad_w = add_list_grad(list_grad_w.copy(), bp_list[0])
      list_grad_b = add_list_grad(list_grad_b.copy() , bp_list[1])
      if((image_no + 1) % batch_size == 0):
        list_grad_w.reverse()
        list_grad_b.reverse()
        theta = add_list_grad_eta(theta.copy() , list_grad_w, eta)
        b = add_list_grad_eta(b.copy(), list_grad_b , eta)
        list_grad_w.clear()
        list_grad_b.clear()
      # print(theta)
      image_no += 1

    # Update theta for last few images
    if (len(x_train) % batch_size != 0):
      list_grad_w.reverse()
      list_grad_b.reverse()
      theta = add_list_grad_eta(theta.copy() , list_grad_w, eta)
      b = add_list_grad_eta(b.copy(), list_grad_b , eta)
    print(loss/len(x_train))
    # list_grad_w.reverse()
    # list_grad_b.reverse()
    # theta = add_list_grad_eta(theta , list_grad_w, eta)
    # b = add_list_grad_eta(b, list_grad_b , eta)
  return(theta,b)

In [ ]:
# def train_NN():


In [ ]:
#hyperparameters

LEARNING_RATE = 0.001
ACTIVATION = "sigmoid"
INITIALIZER = "xavier"
OPTIMIZER = "sgd"
BATCH_SIZE = 512
EPOCHS = 40
L2_lambda = 0.0005
LAYER_DIMS = [784,64,32,64,10]

LOSS = 'categorical_crossentropy'


learning_rate = 0.01
activation = "sigmoid"
initializer = "xavier"
optimizer = "sgd"
batch_size = 32
epochs = 10
loss_fun = "cross_entropy"
num_layers = 2
num_neurons = 32

In [ ]:
def neural_network(x_train,y_train,x_test,y_test,eta,activation,initializer,optimizer,batch_size,epochs,loss_fun):
  if(initializer == "xavier"):
    theta = initilize_theta_xavier(num_layers,num_neurons)
    b = initilize_bias(num_layers,num_neurons)
  elif (initializer == "random"):
    theta = initilize_theta_random(num_layers,num_neurons)
    b = initilize_bias(num_layers,num_neurons)
  
  # invoking training fun
  if(optimizer == "sgd"):
    theta,b = SGD(theta,b,epochs,eta,activation,x_train,y_train,num_neurons, num_layers,batch_size)


  return(theta,b)
 


In [ ]:
theta , b =neural_network(X_train,Y_train,x_test,y_test,learning_rate,activation,initializer,optimizer,batch_size,epochs,loss_fun)

1
0.7321355820127212
2
0.446545657506687
3
0.4010977821052966
4
0.37478364311363876
5
0.35628150932632796
6
0.34201627765902176
7
0.33051204749600577
8
0.32082530639015655
9
0.3123108727884669
10
0.30472041354605384


In [ ]:
 # testing accuracy
wrong_count=image_pridiction(x_test, y_test, theta, b, num_layers,activation)
print(wrong_count)
accuracy = (len(x_test) - wrong_count) * 100 / len(x_test)
print(accuracy)

1313
86.87
